In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [ ]:
!pip install langchain_google_genai
!pip install langchain_community
!pip install pinecone-client
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Gemma patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.6 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata


# Initialize the Google Generative AI embeddings model
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    task_type="retrieval_document",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
)

def get_embeddings(text: str):
    """
    Get embeddings for a given text using the Google Generative AI embedding model.
    """
    try:
        # Generate embeddings
        embeddings = embedding_model.embed_query(text)
        return embeddings
    except Exception as e:
        print(f"Failed to generate embeddings: {e}")
        raise e

# Create a callable embedding function
def embedding_function(text: str):
    """
    A callable function that takes a text input and returns its embedding.
    """
    return get_embeddings(text)

In [ ]:
! pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inference-1.1.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 3.1.0 whic

In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore



pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
index_name = "sarathi-vectorstore"

index = pc.Index(index_name)

vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [ ]:

import google.generativeai as genai




def rag_query(question: str, k: int = 5):
    """
    Perform RAG query: retrieve relevant docs + generate answer.
    """
    relevant_docs = vector_store.similarity_search(question, k=k)

    return relevant_docs

In [ ]:
context = rag_query("छात्रबृत्तिको लागि सिफारिस पाऊँ")
print(context)

[Document(id='60bdedb0-8d26-4760-a05b-74141e5d160e', metadata={'createdAt': '2025-01-24T06:16:17.967Z', 'page': 7.0, 'source': 'Nepali text'}, page_content='\n\t\t\t\t८ आर्थिक अवस्था कमजोर सिफारिस पाऊँ\n\t\t\t\tसंलग्न गर्नु पर्ने कागजातहरु :\n\t\t\n\t\t\t\t१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n\t\t\t\t२) बसाईसराइ भए बसाइसराइ प्रमाणपत्रको प्रतिलिपि\n\t\t\t\t३) विषय प्रमाणित गर्ने कागजात भए सोको प्रतिलिपि\n\t\t\t\t'), Document(id='765a6f4e-6962-45a8-8666-fd93196449ba', metadata={'createdAt': '2024-12-28T10:02:07.633Z', 'page': 0.0, 'source': 'Nepali test'}, page_content='\n\t\t\t\t२ मोही लगत कट्टाको सिफारिस माग्नु\n\t\t\t\tसंलग्न गर्नुपर्ने कागजातहरु:\n\t\t\t\t१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n\t\t\t\t२) जग्गा धनी प्रमाण पुर्जाको प्रतिलिपि\n\t\t\t\t३) जग्गाको प्रमाणित नापी नक्सा\n\t\t\t\t४) नाता प्रमाणित प्रमाणपत्रको प्रतिलिपि\n\t\t\t\t५) जग्गाको श्रेस्ता र फिल्डबुकका प्रमाणित प्रतिलिपि\n\t\t\t\t६) सर्जमिन मुचुल्का गरी बुझनु पर्ने भएमा साक्षी बस्नेको नागरिकता प्रमाणपत्र

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Context:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
    prompt.format(
        """Given the context, you have to reply it to the user in NEPALI letters.
        Don't hallucinate or repeat the same answer or give any more information,
        give the exact answer based on the question in input.""", # instruction
        [context[i].page_content.replace('\t', '') for i in range(0, len(context))], # context
        "छात्रवृत्ति प्राप्त गर्नका लागि सिफारिस", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

print(prompt.format(
        """Given the context, you have to reply it to the user in NEPALI letters.
        Don't hallucinate or repeat the same answer or give any more information,
        give the exact answer based on the question in input.""", # instruction
        [context[i].page_content.replace('\t', '') for i in range(0, len(context))], # context
        "छात्रवृत्ति प्राप्त गर्नका लागि सिफारिस", # input
        "", # output - leave this blank for generation!
    ))


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the context, you have to reply it to the user in NEPALI letters. 
        Don't hallucinate or repeat the same answer or give any more information, 
        give the exact answer based on the question in input.

### Context:
['\n८ आर्थिक अवस्था कमजोर सिफारिस पाऊँ\nसंलग्न गर्नु पर्ने कागजातहरु :\n\n१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n२) बसाईसराइ भए बसाइसराइ प्रमाणपत्रको प्रतिलिपि\n३) विषय प्रमाणित गर्ने कागजात भए सोको प्रतिलिपि\n', '\n२ मोही लगत कट्टाको सिफारिस माग्नु\nसंलग्न गर्नुपर्ने कागजातहरु:\n१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n२) जग्गा धनी प्रमाण पुर्जाको प्रतिलिपि\n३) जग्गाको प्रमाणित नापी नक्सा\n४) नाता प्रमाणित प्रमाणपत्रको प्रतिलिपि\n५) जग्गाको श्रेस्ता र फिल्डबुकका प्रमाणित प्रतिलिपि\n६) सर्जमिन मुचुल्का गरी बुझनु पर्ने भएमा साक्षी बस्नेको नागरिकता प्रमाणपत्रको प्रतिलिपि\n', '\n

In [ ]:
outputs = model.generate(**inputs, max_new_tokens = 150, use_cache = True)
outputs = tokenizer.batch_decode(outputs)
print(outputs[0])

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the context, you have to reply it to the user in NEPALI letters. 
        Don't hallucinate or repeat the same answer or give any more information, 
        give the exact answer based on the question in input.

### Context:
['\n८ आर्थिक अवस्था कमजोर सिफारिस पाऊँ\nसंलग्न गर्नु पर्ने कागजातहरु :\n\n१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n२) बसाईसराइ भए बसाइसराइ प्रमाणपत्रको प्रतिलिपि\n३) विषय प्रमाणित गर्ने कागजात भए सोको प्रतिलिपि\n', '\n२ मोही लगत कट्टाको सिफारिस माग्नु\nसंलग्न गर्नुपर्ने कागजातहरु:\n१) निवेदकको नागरिकता प्रमाणपत्रको प्रतिलिपि\n२) जग्गा धनी प्रमाण पुर्जाको प्रतिलिपि\n३) जग्गाको प्रमाणित नापी नक्सा\n४) नाता प्रमाणित प्रमाणपत्रको प्रतिलिपि\n५) जग्गाको श्रेस्ता र फिल्डबुकका प्रमाणित प्रतिलिपि\n६) सर्जमिन मुचुल्का गरी बुझनु पर्ने भएमा साक्षी बस्नेको नागरिकता प्रमाणपत्रको प्रतिलिपि\n'